### RSRS择时实现代码：

In [1]:
#初始化函数 
def initialize(context):
    set_benchmark('000300.XSHG')
    # 用真实价格交易
    set_option('use_real_price', True)
    # 打开防未来函数
    set_option("avoid_future_data", True)
    # 将滑点设置为0
    set_slippage(FixedSlippage(0))
    # 设置交易成本万分之三
    set_order_cost(OrderCost(open_tax=0, close_tax=0.001, open_commission=0.0003, close_commission=0.0003, close_today_commission=0, min_commission=5),type='fund')
    # 过滤order中低于error级别的日志
    log.set_level('order', 'error')
    #选股参数
    g.stock_num = 4 #持仓数
    # 设置交易时间，每天运行
    g.ref_stock = '000300.XSHG' #用ref_stock做择时计算的基础数据,上证
    g.N = 18 # 计算最新斜率slope，拟合度r2参考最近N天
    g.M = 600 # 计算最新标准分zscore，rsrs_score参考最近M天
    g.score_threshold = 0.7 # rsrs标准分指标阈值
    g.mean_day = 30 #计算结束ma收盘价，参考最近mean_day
    g.mean_diff_day = 2 #计算初始ma收盘价，参考(mean_day + mean_diff_day)天前，窗口为mean_diff_day的一段时间
    g.slope_series = initial_slope_series()[:-1] # 除去回测第一天的slope，避免运行时重复加入
    g.sold_stock = {}
    g.buy_again = 5
    #原版是每周调仓一次
    #run_weekly(my_trade, weekday=1, time='9:30', reference_security='000300.XSHG')
    #run_daily(before_trade, time='00:01',reference_security='000300.XSHG')
    #run_daily(before_market_open, time='before_open', reference_security='000300.XSHG')
    #run_daily(buy, time='every_bar', reference_security='000300.XSHG')
    #run_daily(sell, time='every_bar', reference_security='000300.XSHG')

In [2]:
# 最小二乘拟合
def get_ols(x, y):
    slope, intercept = np.polyfit(x, y, 1)
    r2 = 1 - (sum((y - (slope * x + intercept))**2) / ((len(y) - 1) * np.var(y, ddof=1)))
    return (intercept, slope, r2) # r2为这里的决定系数

def initial_slope_series():
    data = attribute_history(g.ref_stock, g.N + g.M, '1d', ['high', 'low'])
    return [get_ols(data.low[i:i+g.N], data.high[i:i+g.N])[1] for i in range(g.M)]

# 因子标准化
def get_zscore(slope_series):
    mean = np.mean(slope_series)
    std = np.std(slope_series)
    return (slope_series[-1] - mean) / std

# 只看RSRS因子值作为买入、持有和清仓依据，前版本还加入了移动均线的上行作为条件
def get_timing_signal(context,stock):
    g.mean_diff_day = 5
    # 30+5 天
    close_data = attribute_history(g.ref_stock, g.mean_day + g.mean_diff_day, '1d', ['close'])
    high_low_data = attribute_history(g.ref_stock, g.N, '1d', ['high', 'low'])
    close_data_001 = get_bars('000001.XSHG', 10, '1d', 'close', include_now=True)['close']
    price_now = get_bars('000001.XSHG', 1, '1m', 'close', include_now=True)['close']
    ma_old = close_data_001.mean()

    # 这两句同上面的功能相同，愿意测试的可以试试，与avoid_future_data相互矛盾，只能取二者中的一个
    # close_data = get_price(g.ref_stock, end_date=context.current_dt-datetime.timedelta(1),count=g.mean_day + g.mean_diff_day,fields=['close'])
    # high_low_data = get_price(g.ref_stock, end_date=context.current_dt-datetime.timedelta(1),count=g.N, fields=['high', 'low'])

    intercept, slope, r2 = get_ols(high_low_data.low, high_low_data.high)
    g.slope_series.append(slope)
    rsrs_score = get_zscore(g.slope_series[-g.M:]) * r2
    if rsrs_score > g.score_threshold: return "BUY"
    elif rsrs_score < -g.score_threshold or  price_now <= ma_old*1.002 : return "SELL"#close_data_001[-1]
    else: return "KEEP"

### 大盘均线择时实现代码：

In [ ]:
def get_siginal(context):
    #或者用当前日和昨日的5日均线值来比较，作为是否买入的信号
    close_data_5 = get_bars('000001.XSHG', 5, '1d', 'close', include_now=True)['close']
    price_open = get_bars('000001.XSHG', 1, '1d', 'open', include_now=True)['open']
    price_now = get_bars('000001.XSHG', 1, '1m', 'close', include_now=True)['close']
    #if price_now < close_data_5.mean() and price_now < close_data_10.mean():
    #close_today_3 = get_bars('000001.XSHG', 3, '1d', 'close', include_now=True)['close']
    #close_yesterday_3 = get_bars('000001.XSHG', 3, '1d', 'close', include_now=False)['close']
    if price_now > price_open:
        return 'buy'
    elif price_now > close_data_5.mean():
        return 'buy'
    elif price_open < close_data_5.mean():
        return 'sell'
    return 'keep'